### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Fall 2018 ###

# Bakery---> Food Cart Optimization #

#### Nicholas Timmons (ntimmons@wisc.edu -  906 982 9696)


### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## Introduction ##

Consider a bakery that has had a successful storefront location in for several years. They have just decided that they would like to expand their business into food carts in the downtown area, in order to capitalize on the increased foot traffic there. The problem is that their storefront is quite large, and any food cart they operate will have much less space on the counter to  display their products. In order to decide which products they should sell at their food carts, they decide to hire a programmer to write an optimization model that maximize profit while only selling P products. They also decide that they want to see which items would be most profitable during every hour that they are open- based on their storefront sales.

In order to solve an example of this problem, we have taken a large(21,293 samples) dataset from the internet([kaggle.com](https://www.kaggle.com/sulmansarwar/transactions-from-a-bakery). This dataset contains real transactions from a bakery, and records the date and time of the transaction, and the name of the item solve in that transaction. In order to get this data into a form that we can use, we must do some parsing.

Since we don't care about the date in this problem, we must first parse the time and see how many sales are made at each hour. We disregard hours where there are not meaningful sales(relative to the size of the dataset) and create a list of times that **are** meaningful to us. Then we parse the items sold, and disregard items that are not meaningful to us(ie. there are very few sales done in the entire dataset) and create a list of these item names. From here we are ready to create and run the optimization model on this data. 

**NOTE: The dataset did not provide price points for the items sold, so we supplied arbritrary prices that seemed to fit the item being sold**

In what follows, we will lay out the mathematical model that helps us solve this problem, the variables involved, and the results of the model.


## Mathematical model ##

In order to solve this problem and arrive at the P items that should be sold, we have to formulate
a MIP(Mixed Integer Program). We need this sort of model because we need the decisions
variables that represent the items that **could** be sold at the food carts to be either 1 or 0
after we run the solver. This is because we just need to know if we should sell an item(1) or 
not(0). The maximization objective is not restricted to being an integer, because we have to use the price of the product, which is not necessarily an integer. 

Since we are trying to optimize products sold at each hour of operation, we will need to create models, variables, and constraints iteratively over these hours. For each hour we iterate over the data and check if the timestamp is the same as the hour we are currently optimizing. If it is, we add that item to the current model. 

We create a constraint that sums all of the possible items to sell- z[1] - z[37] in this case. We restrict this sum to being less than or equal to the total number of items the bakery would like to sell- in this example we solve for five items. 

We then create an objective function that sums each decision variable z times the price of that item times the number of that item sold during that hour. This will maximize the profit of each item, and the model will pick the five items that maximize the total profit at the food cart. 

Here is a mathematical look at our model:



$$\begin{aligned}
  {\text{maximize}}\qquad& \sum_{n=1}^{P} Z_i  J_i && i=1,\dots,P && Z_i \in [1,0]\\
    \text{subject to:}\qquad& \sum_{n=1}^{P} Z_i \le M  && i=1,\dots,P && Z_i \in [1,0]\\
    \end{aligned}$$


Here P is the number of products we are considering, Z's are the decision variables, J's are the price of that item, and M is the maximum number of items that we can sell.

In [3]:
## Solution ##
using NamedArrays, JuMP, Cbc

raw = readcsv("bakery.csv")#read in the dataset
(m,n) = size(raw)#get dimensions of the dataset

#This array contains the names of foods that have meaningful data in the dataset
#ie. there was more than one or a few of these items sold
g = String["Spanish Brunch", "Baguette", "Tea", "Tartine", "Jammie Dodgers", "Bread", "Muffin",
    "Art Tray", "Hot chocolate", "Scone", "Coffee", "Scandinavian", "Salad", "Fudge", "Soup",
    "Cookies", "Jam", "Cake", "Eggs", "Medialuna", "Brownie", "Bakewell", "Sandwich", "Alfajores",
    "Focaccia", "Tiffin", "Juice", "Smoothies", "Frittata", "Pastry", "Truffles", "Chicken Stew",
    "The Nomad", "Mineral water", "Toast", "Vegan mincepie", "Farm House"]

#get raw data for dates, times, and items
n_date = 2:m
n_time = 2:m
n_items = 2:m

#get the actual dates, times, and items
date = raw[n_date, 1][:]
time = raw[n_time, 2][:]
items = raw[n_items, 4][:]


for i = 10:17 # the times od the day that had meaningful sales
    d = Dict() #empty dictionary to fill for item sales at each time of day
    for j in 1:21293 #length of the dataset
        x = split(time[j], r"[:]")#split time fields
        if parse(x[1]) == i #check if current time field matches the time we are currently looking at
            if items[j] in g # if this item is in g
                if !(haskey(d, items[j])) #if there is not already an entry in the dictionary, add one and set the value to 1
                    d[items[j]] = 1
                    else # if there is already and entry in the dictionary, increment its value
                    d[items[j]] = d[items[j]] +1
                end
            end
        end
    end
    
    m = Model(solver=CbcSolver())
    @variable(m, z[1:37], Bin) #a variable for each item sold, binary because we just want to see if an 
    #item should be sold or not
    
    #constrain the variables so that only 5 can be active
    @constraint(m, z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] +
    z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] +
    z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] +
    z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] +
    z[36] + z[37] <= 5)
    
    #maximize the active constraints * the sale price * number of items sold
    @objective(m, Max, get(d, g[1], 0)*11z[1] + get(d, g[2], 0)*5z[2] + get(d, g[3], 0)*1.5z[3]
        + get(d, g[4], 0)*4.5z[4] + get(d, g[5], 0)*6z[5] + get(d, g[6], 0)*z[6] + 
        get(d, g[7], 0)*2.5z[7] + get(d, g[8], 0)*8z[8] + get(d, g[9], 0)*3z[9] + 
        get(d, g[10], 0)*5z[10] + get(d, g[11], 0)*3z[11] + get(d, g[12], 0)*3z[12] +
        get(d, g[13], 0)*7z[13] + get(d, g[14], 0)*6z[14] + get(d, g[15], 0)*3.5z[15] +
        get(d, g[16], 0)*2z[16] + get(d, g[17], 0)*z[17] + get(d, g[18], 0)*12z[18] +
        get(d, g[19], 0)*4z[19] + get(d, g[20], 0)*5z[20] + get(d, g[21], 0)*3.5z[21] +
        get(d, g[22], 0)*8z[22] + get(d, g[23], 0)*6.5z[23] + get(d, g[24], 0)*7z[24] +
        get(d, g[25], 0)*6z[25] + get(d, g[26], 0)*4z[26] + get(d, g[27], 0)*3z[27] +
        get(d, g[28], 0)*4z[28] + get(d, g[29], 0)*5.5z[29] + get(d, g[30], 0)*2.5z[30] +
        get(d, g[31], 0)*9z[31] + get(d, g[32], 0)*8z[32] + get(d, g[33], 0)*7z[33] +
        get(d, g[34], 0)*2z[34] + get(d, g[35], 0)*3z[35] + get(d, g[36], 0)*8z[36] + 
        get(d, g[37], 0)*6z[37])
    status = solve(m)
    println()
    x = getvalue(z)
    println("From ", i, ":00 to ", i, ":59 the optimal choice of items is:")
    for i in 1:length(x)
        if x[i] != 0
            println(g[i])
        end
    end
end



From 10:00 to 10:59 the optimal choice of items is:
Bread
Coffee
Cake
Medialuna
Pastry

From 11:00 to 11:59 the optimal choice of items is:
Bread
Coffee
Cake
Medialuna
Farm House

From 12:00 to 12:59 the optimal choice of items is:
Spanish Brunch
Bread
Coffee
Cake
Sandwich

From 13:00 to 13:59 the optimal choice of items is:
Spanish Brunch
Coffee
Soup
Cake
Sandwich

From 14:00 to 14:59 the optimal choice of items is:
Coffee
Cake
Sandwich
Alfajores
Truffles

From 15:00 to 15:59 the optimal choice of items is:
Tea
Coffee
Cake
Sandwich
Alfajores

From 16:00 to 16:59 the optimal choice of items is:
Bread
Hot chocolate
Coffee
Cake
Alfajores

From 17:00 to 17:59 the optimal choice of items is:
Tea
Coffee
Cake
Medialuna
Alfajores


## Results and discussion ##

Above we see the output of the model for each hour of the day. We see that there some items
like bread and coffee that should be sold at almost every hour of the day, and others that are only popular at one hour of the day, like hot chocolate. 

It is important to note that these results only show the most popular items that were sold at the storefront at those times. We cannot necessarily extrapolate these findings onto food cart sales, because there is likely a very different customer base, and there are other things to take into consideration when considering foods to sell. 

At a food cart, there is limited ability to make or store certain items that would require extra equipment. Would this extra space requirement make any items infeasible or unprofitable to sell? it is certainly possible. Also, since we are considering swapping out items at each hour of the day, then we might also consider the cost of changing the display, and the space requirement to store these additional items behind the counter. 

These results give a very good basis for choosing items to sell, but are not necessarily the best when you consider these additional constraints. Without knowing these constraints, we cannot do any better than our current model.


## Conclusion ##

Given the output of our model, we can conclude that we should **definitely** sell coffee and cake at every hour of business. 

Besides these popular items, we can draw other conclusions. We see that sandwiches are very popular during the middle of the day, so we could reasonable plan on selling them for lunch, but possibly changing that item's spot for something else during the morning and nighttime. 

In a future model that expands on this one, we could further modify the constraints so that we take into account the cost of changing out an item, and the storage space behind the counter. 

Is there a possibility of selling out of a certain item because we can only store a certain amount of it behind the counter? Then we might consider the maximum amount of an item that can be stored, and the corresponding maximum profit of that item, and instead of summing the decision variables times their price, we could sum the decision variables times the total profit of that item. This would create a better output if certain items will take up so much space that their profit does not make it worthwhile selling.
